<a href="https://colab.research.google.com/github/sineka232/data_science/blob/main/Fake_news_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers datasets shap gradio scikit-learn accelerate --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.8 MB/s eta 0:00:00


In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"


In [3]:
import pandas as pd

train_data = {
    "text": [
        "Government approves new education policy",
        "NASA discovers water on the moon",
        "Actor caught in controversy is confirmed fake news",
        "COVID-19 can be cured by drinking hot water",
        "Scientists warn about global warming",
        "Local school wins national award for innovation"
    ],
    "label": [0, 0, 1, 1, 0, 0]
}

val_data = {
    "text": [
        "New tax reforms announced today",
        "Eating garlic can prevent coronavirus"
    ],
    "label": [0, 1]
}

test_data = {
    "text": [
        "WHO confirms new virus variant",
        "Drinking alcohol kills viruses"
    ],
    "label": [0, 1]
}

pd.DataFrame(train_data).to_csv("train.csv", index=False)
pd.DataFrame(val_data).to_csv("val.csv", index=False)
pd.DataFrame(test_data).to_csv("test.csv", index=False)

print("Created train.csv, val.csv, test.csv")


Created train.csv, val.csv, test.csv


In [4]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files={
        "train": "train.csv",
        "validation": "val.csv",
        "test": "test.csv"
    }
)
dataset


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})

In [5]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding=True, max_length=128)

tokenized = dataset.map(tokenize_fn, batched=True)
# keep only columns the Trainer expects
tokenized = tokenized.remove_columns([c for c in tokenized["train"].column_names if c not in ["input_ids","attention_mask","label"]])
tokenized = tokenized.rename_column("label", "labels")
tokenized.set_format(type="torch")
tokenized


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 6
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 2
    })
})

In [7]:
from transformers import AutoModelForSequenceClassification
import torch

# Use tiny 4-layer BERT (loads instantly, perfect for Colab)
model_name = "prajjwal1/bert-tiny"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

model.to(device)


Device: cpu


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/model_out",
    eval_strategy="epoch",            # works with older/newer Transformers
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=1
)


⚙️  Running in WANDB offline mode


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [9]:
from transformers import Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score
import numpy as np

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {"accuracy": float(accuracy_score(labels, preds))}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


/tmp/ipython-input-4199400114.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.614718,1.000000
2,No log,0.613911,1.000000
3,No log,0.613600,1.000000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=3, training_loss=0.7129911581675211, metrics={'train_runtime': 1.7622, 'train_samples_per_second': 10.214, 'train_steps_per_second': 1.702, 'total_flos': 580652280.0, 'train_loss': 0.7129911581675211, 'epoch': 3.0})

In [10]:
trainer.evaluate(tokenized["test"])


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.7259948253631592,
 'eval_accuracy': 0.5,
 'eval_runtime': 0.1031,
 'eval_samples_per_second': 19.407,
 'eval_steps_per_second': 9.704,
 'epoch': 3.0}

In [11]:
import shap
import numpy as np

# Ensure model is in eval mode and on correct device
model.eval()

def shap_model_predict(texts):
    """
    texts: list of strings
    returns: numpy array of shape (len(texts), num_labels) with probabilities
    """
    # Tokenize and move tensors to model device
    tokens = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    tokens = {k: v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
        probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
    return probs

# Create SHAP explainer with the wrapper. Passing tokenizer as the "masker" works reasonably here.
explainer = shap.Explainer(shap_model_predict, tokenizer)
print("SHAP explainer ready")


SHAP explainer ready


In [12]:
import torch
from IPython.display import HTML
import shap

def classify_text(text):
    probs = shap_model_predict([text])[0]
    label = "🔴 FAKE NEWS" if np.argmax(probs) == 1 else "🟢 REAL NEWS"
    return label, float(probs[1]), float(probs[0])

def explain_text_to_html(text):
    # produce shap values and render as HTML
    shap_values = explainer([text])
    # shap.plots.text returns a matplotlib/JS object; using display=False returns HTML-like object in many versions
    html_obj = shap.plots.text(shap_values, display=False)
    # If shap returns an object with .data or .html attribute, convert it. We try several fallbacks.
    try:
        # shap >= some versions return a matplotlib/plotly-like object
        return html_obj
    except Exception:
        # fallback: return pretty plain highlighting
        tokens = tokenizer(text, return_tensors="pt")["input_ids"][0]
        tokens = tokenizer.convert_ids_to_tokens(tokens)
        return "<p>" + " ".join(tokens) + "</p>"


In [18]:
# ---- Cell 12 ----
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import pickle

# Sample dataset (we will create small one)
data = {
    "text": [
        "Government announces new health policy for senior citizens.",
        "NASA discovered water on the moon according to scientists.",
        "WhatsApp will start charging ₹2 for every message sent.",
        "A man claimed he travelled to the future using a microwave.",
        "The Indian economy grew by 7.5% last quarter.",
        "Schools will be closed for 3 years due to solar eclipse.",
    ],
    "label": [1, 1, 0, 0, 1, 0]  # 1 = real, 0 = fake
}

df = pd.DataFrame(data)

# Split
X_train, X_test, y_train, y_test = train_test_split(df.text, df.label, test_size=0.2, random_state=42)

# Create model pipeline
model = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", LogisticRegression())
])

model.fit(X_train, y_train)

# Save model
with open("fake_news_model.pkl", "wb") as f:
    pickle.dump(model, f)

def classify_news(text):
    if not text:
        return "Please enter text"
    pred = model.predict([text])[0]
    prob = model.predict_proba([text])[0]

    label = "🟢 REAL NEWS" if pred == 1 else "🔴 FAKE NEWS"
    return f"{label}\nReal: {prob[1]:.3f} | Fake: {prob[0]:.3f}"


In [ ]:
# ---- Cell 13 ----
import gradio as gr

def predict_ui(text):
    return classify_news(text)

demo = gr.Interface(
    fn=predict_ui,
    inputs=gr.Textbox(label="Enter News Text", lines=5, placeholder="Paste news text..."),
    outputs=gr.Textbox(label="Prediction"),
    title="📰 Fake News Detector (Offline Version)",
    description="This model runs fully offline without HuggingFace."
)

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://40f9c8a412a1bbdc2c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
